# Lecture 9-1

# Intro to SQL

## Week 9 Monday

## Miles Chen, PhD

# Do not try to set up your own SQL Server

There is a big difference between setting up a SQL server and learning how to write a few queries.

We will learn how to write a few queries.

We will not learn how to set up a server and design a database. Most likely, if you go into the workforce and they list SQL as a requirement, they need someone who can write queries. The person who sets up, designs, and maintains the server will be a database administrator.

# Learning more

Recommended exercises: <https://www.w3resource.com/sql-exercises/>

Another place to just practice SQL queries: <http://sqlfiddle.com>

# sqlalchemy

sqlalchemy allows us to connect and interact with databases from within Python

Most of your SQL experience will be with connecting to a database that already exists. Most data analysts / data scientists are not the data base administrator and this is not a database administration course.

You can download the chinook database file from the chinook database github:

https://github.com/lerocha/chinook-database/tree/master/ChinookDatabase/DataSources

In [2]:
import pandas as pd

In [4]:
from sqlalchemy import create_engine

In [5]:
# create_engine creates a connection to an existing database
# I have 'Chinook_sqlite.sqlite' downloaded into my folder, and python
# connects to this database
engine = create_engine('sqlite:///Chinook_Sqlite.sqlite')

In [6]:
from sqlalchemy import inspect
insp = inspect(engine) # creates an inspector

In [7]:
# Use the inspector to get table names
# Save the table names to a list: table_names
table_names = insp.get_table_names()

# Print the table names to the shell
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


## Basics

SQL is generally not case sensitive. Convention, however, puts SQL commands in ALL-CAPS and then leaves column and variable names in the same case as they appear in the table.

semi-colons are not required to terminate SQL queries, but their usage is recommended.

### `SELECT`

`SELECT` is used to select variables from a given table. To select all columns, use `*`

### `FROM`

`FROM` specifies which table to select from.

Once you create the database engine with `sqlalchemy`, we can begin executing SQL queries by establishing a connection with the database.

In [8]:
from sqlalchemy import text

In [9]:
# Open engine connection
con = engine.connect()

# Perform query and store results in rs
# this will select all columns from the Album table
rs = con.execute(text('SELECT * FROM Album;'))

# Fetch all results of the query and save to DataFrame
df = pd.DataFrame(rs.fetchall())

# Close the connection to the engine
con.close()

# Print head of query results
print(df.head(10))
print(rs.keys())

   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3
5        6                     Jagged Little Pill         4
6        7                               Facelift         5
7        8                         Warner 25 Anos         6
8        9         Plays Metallica By Four Cellos         7
9       10                             Audioslave         8
RMKeyView(['AlbumId', 'Title', 'ArtistId'])


Instead of having to open and close the engine connection, we can use Python's with statement which will automatically open and close the connection for us

In [10]:
# We can write our SQL command across multiple lines
# enclosed in triple quotes
command = '''
SELECT FirstName, LastName, Title 
FROM Employee;
'''

# SELECT chooses the desired columns
# FROM indicates the table to query

with engine.connect() as con:
    rs = con.execute(text(command))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

  FirstName  LastName                Title
0    Andrew     Adams      General Manager
1     Nancy   Edwards        Sales Manager
2      Jane   Peacock  Sales Support Agent
3  Margaret      Park  Sales Support Agent
4     Steve   Johnson  Sales Support Agent
5   Michael  Mitchell           IT Manager
6    Robert      King             IT Staff
7     Laura  Callahan             IT Staff


Pandas offers functionality to directly query a SQL database using an existing engine

In [11]:
# we can use the same command as earlier:
query = text(command)
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df

,FirstName,LastName,Title
0,Andrew,Adams,General Manager
1,Nancy,Edwards,Sales Manager
2,Jane,Peacock,Sales Support Agent
3,Margaret,Park,Sales Support Agent
4,Steve,Johnson,Sales Support Agent
5,Michael,Mitchell,IT Manager
6,Robert,King,IT Staff
7,Laura,Callahan,IT Staff


## `ORDER BY`

ORDER BY is SQL's version of sort

<https://www.w3schools.com/sql/sql_orderby.asp>

```
SELECT column1, column2, ...
FROM table_name
ORDER BY column1, column2, ... ASC|DESC;
```

In [12]:
command = '''
SELECT * 
FROM Employee 
ORDER BY Birthdate DESC;
'''
pd.read_sql_query(text(command), conn)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
1,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
2,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
3,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
6,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
7,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com


## `WHERE`

Filter row selection with WHERE. (similar to using if as a boolean mask)

SQL uses single equal sign = for comparison

In [13]:
command = '''
SELECT * 
FROM Employee 
WHERE EmployeeId >= 6 AND Title = 'IT Staff'
ORDER BY BirthDate;
'''
pd.read_sql_query(text(command), conn)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com


### `JOIN` and `LIMIT`

We can look at data across multiple tables using a `JOIN`

`LIMIT` acts like "head()", and limits the number of entries it returns

In [14]:
command = '''
SELECT * 
FROM Album
LIMIT 10;
'''
pd.read_sql_query(text(command), conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,9,Plays Metallica By Four Cellos,7
9,10,Audioslave,8


In [15]:
command = '''
SELECT * 
FROM Artist
LIMIT 10;
'''
pd.read_sql_query(text(command), conn)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
5,6,Antônio Carlos Jobim
6,7,Apocalyptica
7,8,Audioslave
8,9,BackBeat
9,10,Billy Cobham


`JOIN`

`INNER JOIN` is a specific type of join. It keeps only rows where the key exists in both tables. If one table is missing an entry that exists in the other table, the entry will not be returned.

When using a `JOIN`, specify the name of the table that is being joined and the columns used to match the rows. Columns are specified with dot notation. `TableName.ColumnName`

In [16]:
command = '''
SELECT * 
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
LIMIT 10;
'''
pd.read_sql_query(text(command), conn)

,AlbumId,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,Accept
2,3,Restless and Wild,2,2,Accept
3,4,Let There Be Rock,1,1,AC/DC
4,5,Big Ones,3,3,Aerosmith
5,6,Jagged Little Pill,4,4,Alanis Morissette
6,7,Facelift,5,5,Alice In Chains
7,8,Warner 25 Anos,6,6,Antônio Carlos Jobim
8,9,Plays Metallica By Four Cellos,7,7,Apocalyptica
9,10,Audioslave,8,8,Audioslave


In [17]:
# you can rename columns using `AS`
command = '''
SELECT Title AS "Album Title", Name AS "Artist Name"
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
LIMIT 10;
'''
pd.read_sql_query(text(command), conn)

,Album Title,Artist Name
0,For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Accept
2,Restless and Wild,Accept
3,Let There Be Rock,AC/DC
4,Big Ones,Aerosmith
5,Jagged Little Pill,Alanis Morissette
6,Facelift,Alice In Chains
7,Warner 25 Anos,Antônio Carlos Jobim
8,Plays Metallica By Four Cellos,Apocalyptica
9,Audioslave,Audioslave


`GROUP BY` can be used to create groups to help calculate summary values

`COUNT()` is one function that can be used to calculate summary values. Other summary functions include `SUM()` and `AVG()`

In [18]:
command = """
SELECT Artist.ArtistId, Name, COUNT(AlbumId) AS album_count,
  AVG(AlbumId) AS avg_id, SUM(AlbumID) as sum
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.ArtistId
LIMIT 10;
"""
pd.read_sql_query(text(command), conn)

,ArtistId,Name,album_count,avg_id,sum
0,1,AC/DC,2,2.500000,5
1,2,Accept,2,2.500000,5
2,3,Aerosmith,1,5.000000,5
3,4,Alanis Morissette,1,6.000000,6
4,5,Alice In Chains,1,7.000000,7
5,6,Antônio Carlos Jobim,2,21.000000,42
6,7,Apocalyptica,1,9.000000,9
7,8,Audioslave,3,97.333333,292
8,9,BackBeat,1,12.000000,12
9,10,Billy Cobham,1,13.000000,13


In [19]:
# for comparison with previous table
command = '''
SELECT * 
FROM Album
ORDER BY ArtistId
LIMIT 15;
'''

pd.read_sql_query(text(command), conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,4,Let There Be Rock,1
2,2,Balls to the Wall,2
3,3,Restless and Wild,2
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,34,Chill: Brazil (Disc 2),6
9,9,Plays Metallica By Four Cellos,7


In [20]:
# Conditionals on the Group By must be done with 'HAVING'
command = '''
SELECT Artist.ArtistId, Name, COUNT(AlbumId) AS album_count
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.ArtistId
HAVING album_count > 8;
'''
pd.read_sql_query(text(command), conn)

,ArtistId,Name,album_count
0,22,Led Zeppelin,14
1,50,Metallica,10
2,58,Deep Purple,11
3,90,Iron Maiden,21
4,150,U2,10


In [21]:
command = '''
SELECT ArtistId, ArtistId * 2 AS "magic number", Name
From Artist 
LIMIT 10;
'''
pd.read_sql_query(text(command), conn)

,ArtistId,magic number,Name
0,1,2,AC/DC
1,2,4,Accept
2,3,6,Aerosmith
3,4,8,Alanis Morissette
4,5,10,Alice In Chains
5,6,12,Antônio Carlos Jobim
6,7,14,Apocalyptica
7,8,16,Audioslave
8,9,18,BackBeat
9,10,20,Billy Cobham


## Table previews

In [22]:
command = '''
SELECT * 
FROM Album
LIMIT 5;
'''
pd.read_sql_query(text(command), conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [23]:
command = '''
SELECT * 
FROM Artist
LIMIT 5;
'''
pd.read_sql_query(text(command), conn)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [24]:
command = '''
SELECT * 
FROM Invoice
LIMIT 5;
'''
pd.read_sql_query(text(command), conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [25]:
command = '''
SELECT * 
FROM InvoiceLine
LIMIT 7;
'''
pd.read_sql_query(text(command), conn)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
5,6,2,12,0.99,1
6,7,3,16,0.99,1


In [26]:
command = '''
SELECT * 
FROM Track
LIMIT 6;
'''
pd.read_sql_query(text(command), conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
5,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99


In [27]:
command = '''
SELECT * 
FROM Customer
LIMIT 5;
'''
pd.read_sql_query(text(command), conn)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


# table aliases

Provide a query showing the Invoices of customers who are from Brazil. The resultant table should show the customer's full name, Invoice ID, Date of the invoice and billing country.

In [ ]:
command = '''
SELECT c.FirstName, c.lastname, 
    i.invoiceid, i.invoicedate, i.billingcountry    -- selects the desired columns
FROM customer AS c                     -- provide an alias to the table, so we dont have to type the full name out
    JOIN invoice AS i
    ON c.customerid = i.customerid     -- this is how the tables are linked
WHERE c.country = 'Brazil'
LIMIT 20;                               -- limits how many rows we get back
'''
pd.read_sql_query(text(command), conn)

## `DISTINCT` 

Provide a query showing a unique list of billing countries from the Invoice table.

In [ ]:
command = '''
SELECT DISTINCT billingcountry 
FROM invoice;
'''
pd.read_sql_query(text(command), conn)

## Joining three tables

The following query shows the invoices associated with each sales agent.

The invoice table has no information about employee. But each invoice has a customer and each customer has a support rep (employee). We connect the invoice table with the employee table by connecting them through the customer table

In [ ]:
command = '''
SELECT e.firstname, e.lastname,   -- employee first and last name
       i.*   -- all columns from invoice table 

FROM invoice AS i
    JOIN customer AS c
    ON c.customerid = i.customerid

    JOIN employee AS e
    ON e.employeeid = c.supportrepid
    
ORDER BY e.employeeid;
'''
pd.read_sql_query(text(command), conn)

Following query shows the Invoice Total, Customer name, Country and Sale Agent name for all invoices and customers.

In [ ]:
command = '''
SELECT i.InvoiceId, i.total,
       e.firstname AS 'employee first', 
       e.lastname AS 'employee last', 
       c.firstname AS 'customer first', 
       c.lastname AS 'customer last', 
       c.country
FROM employee AS e
        JOIN customer AS c 
        ON e.employeeid = c.supportrepid
        JOIN invoice AS i 
        ON c.customerid = i.customerid;
'''
pd.read_sql_query(text(command), conn)

How many Invoices were there in 2011? What are the total sales for that year?

In [ ]:
command = '''
SELECT invoiceId, InvoiceDate, total
FROM invoice as i
WHERE i.invoicedate BETWEEN datetime('2011-01-01') AND datetime('2011-12-31');
'''
pd.read_sql_query(text(command), conn)  # result has 83 rows

In [ ]:
command = '''
SELECT invoiceId, InvoiceDate, total
FROM invoice as i
WHERE i.invoiceID BETWEEN 50 AND 60;
'''
pd.read_sql_query(text(command), conn)  # result has 83 rows

In [ ]:
command = '''
SELECT count(i.invoiceId) as 'count',
    sum(i.total) as 'sum'
FROM invoice as i
WHERE i.invoicedate BETWEEN datetime('2011-01-01') AND datetime('2011-12-31');
'''
pd.read_sql_query(text(command), conn)

Count how many orders were made on each day

In [ ]:
command = '''
SELECT i.InvoiceDate, count(i.invoiceId) as 'count'
FROM invoice as i
WHERE i.invoicedate BETWEEN datetime('2011-01-01') AND datetime('2011-12-31')
GROUP BY i.invoiceDate;
'''
pd.read_sql_query(text(command), conn)

Looking at the InvoiceLine table, provide a query that COUNTs the number of line items for each Invoice.

In [ ]:
command = '''
SELECT *
FROM invoiceline
LIMIT 10;
'''
pd.read_sql_query(text(command), conn)

In [ ]:
command = '''
SELECT invoiceid, count(invoicelineid) AS 'Count'
FROM invoiceline
GROUP BY invoiceid
ORDER BY Count DESC;
'''
pd.read_sql_query(text(command), conn)

Find the invoice with the maximum number of Invoiceline IDs most elegant please

CTE Common Table Expression - allows you to query tables that you created as intermediate steps

In [ ]:
command = '''
WITH InvoiceCounts (id, count) 
AS 
(  -- an intermediate table that aggregates the invoicelineIDs 
   -- pretty much the exact same table we generated in previous step
    SELECT invoiceid, count(invoicelineid) AS 'Count'
    FROM invoiceline
    GROUP BY invoiceid
    ORDER BY Count DESC
)

SELECT MAX(count) as Max, MIN(count) as Min
FROM InvoiceCounts;
'''
pd.read_sql_query(text(command), conn)

In [ ]:
command = '''
WITH InvoiceCounts (id, count) 
AS 
(  -- an intermediate table that aggregates the invoicelineIDs 
   -- pretty much the exact same table we generated in previous step
    SELECT invoiceid, count(invoicelineid) AS 'Count'
    FROM invoiceline
    GROUP BY invoiceid
    ORDER BY Count DESC
)

SELECT count, COUNT(id) as "HowMany"
FROM InvoiceCounts
GROUP BY count;
'''
pd.read_sql_query(text(command), conn)

Provide a query that includes the purchased track name AND artist name with each invoice line item.


In [ ]:
command = '''
SELECT i.*, 
    t.name AS 'track', 
    ar.name AS 'artist'
FROM invoiceline AS i
        JOIN track AS t 
            ON i.trackid = t.trackid     -- i links to t
        JOIN album AS al 
            ON t.albumid = al.albumid    -- t links to al
        JOIN artist AS ar 
            ON al.artistid = ar.artistid;  -- al links to ar
'''
pd.read_sql_query(text(command), conn)

In [39]:
# Look up 
# differences betwen LEFT AND RIGHT JOINS
# https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/

One table is of products
Another table is of sales
assume productid is the link

how do find all the products that do not exist in the sales table?

product_table AS p LEFT OUTER JOIN sales_table AS s
 ON p.productid = s.productid
 WHERE s.productid IS NULL